In [2]:
##############
# Constantes #
##############

#@title  { form-width: "400px" }
IS_TEST = True #@param {type:"boolean"}

---

<h3><center>Cargar librerías</center></h3>

---

In [3]:
import sklearn
import seaborn as sns
import pandas as pd
from copy import deepcopy
import statistics as stat
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures, OneHotEncoder, OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

---

<h3><center>Cargar datasets</center></h3>

---

In [4]:
URL_TRAIN_CSV = "https://gitlab.com/villainnumber1/filesml_proyecto_2/-/raw/main/train.csv"
URL_TEST_CSV = "https://gitlab.com/villainnumber1/filesml_proyecto_2/-/raw/main/test.csv"

IS_TEST = True

X = pd.read_csv(URL_TRAIN_CSV, sep=",")
Y = pd.read_csv(URL_TEST_CSV, sep=",")

X

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,43161,3985,4,Lawrencez,40.0,856431180.0,Accountant,97384.44,8167.370000,1.0,...,Good,405.45,32.643281,336.0,No,77.284943,53.714219,High_spent_Large_value_payments,847.172909,Standard
1,40663,49494,2,Mohammedr,39.0,417453942.0,Writer,31369.58,2607.131667,8.0,...,Bad,4259.91,39.375507,144.0,Yes,171.748418,44.990811,High_spent_Large_value_payments,280.855383,Standard
2,111059,2925,6,Alistairn,17.0,499528888.0,Writer,30171.86,2772.321667,6.0,...,Bad,3406.51,34.039021,123.0,Yes,205.403926,40.781489,High_spent_Large_value_payments,247.322615,Standard
3,72139,21358,2,Dinesh Naire,22.0,859449371.0,Doctor,13100.02,1246.668333,7.0,...,Standard,727.29,34.747262,129.0,Yes,41.177132,23.005463,Low_spent_Large_value_payments,298.156832,Standard
4,119841,21221,4,Bakerg,26.0,485239890.0,Doctor,62313.54,5120.795000,6.0,...,Standard,817.64,24.545394,122.0,NM,196.679642,63.346750,Low_spent_Medium_value_payments,152.669902,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,42636,39869,7,Hendrik Sackmannb,50.0,217529489.0,Media_Manager,107761.29,9047.107500,4.0,...,Standard,34.09,41.899992,211.0,Yes,0.000000,150.220682,High_spent_Large_value_payments,984.123010,Standard
79996,114031,38305,2,Alexj,33.0,98861902.0,Scientist,16218.32,1098.526667,6.0,...,Bad,4466.33,24.275710,98.0,NM,64.533114,34.457893,High_spent_Small_value_payments,260.523798,Poor
79997,85991,9976,6,Luciana Lopezi,25.0,745835485.0,Engineer,29374.77,2377.897500,3.0,...,Standard,1189.37,38.439021,241.0,NM,40.264074,22.234178,High_spent_Medium_value_payments,403.980355,Standard
79998,21240,13193,7,Edward Krudyd,15.0,894419091.0,Manager,43782.45,3543.537500,6.0,...,Bad,3468.25,24.233841,85.0,Yes,240.664739,42.839562,Low_spent_Large_value_payments,146.741586,Poor


---

<h3><center>Añadir nuevas columnas</center></h3>

---

In [5]:
##############################
# Util functions for columns #
##############################

def add_column(df, columna, f):
  df[columna] = df.apply(lambda row: f(row), axis=1)

def map_column(self, columna, f):
  df[columna] = [ f(val) for val in df[columna] ]

######
# Para meterle todas las categorias de X e Y al OneHotEncoder.
# Si no da problemas de shape porque X e Y tienen diferentes categorias.
######
def unionColumnas(X, Y, columnName):
  return list(set(X[columnName]).union(set(Y[columnName])))

In [6]:
#######################################
# Separar Type_of_Loan (tested)       #
# x, x, y and z -> [x: 2, y: 1, z: 1] #
#######################################

def countLoans(row, typeOfLoan):
  tol = row["Type_of_Loan"]

  #display(tol)

  parts = [part[4:] if part.startswith("and") else part for part in tol.split(", ") ]

  #display (parts)

  return len([True for part in parts if part == typeOfLoan])>1



for df in X, Y:
  add_column(df, "_Auto Loans",                lambda row: countLoans(row, "Auto Loan"))
  add_column(df, "_Credit-Builder Loans",      lambda row: countLoans(row, "Credit-Builder Loan"))
  add_column(df, "_Debt Consolidation Loans",  lambda row: countLoans(row, "Debt Consolidation Loan"))
  add_column(df, "_Home Equity Loans",         lambda row: countLoans(row, "Home Equity Loan"))
  add_column(df, "_Mortgage Loans",            lambda row: countLoans(row, "Mortgage Loan"))
  add_column(df, "_Payday Loans",              lambda row: countLoans(row, "Payday Loan"))
  add_column(df, "_Personal Loans",            lambda row: countLoans(row, "Personal Loan"))
  add_column(df, "_Student Loans",             lambda row: countLoans(row, "Student Loan"))
  add_column(df, "_Not Specified Loans",       lambda row: countLoans(row, "Not Specified"))

---

<h3><center>Transformación de columnas</center></h3>

---

In [7]:
normalizer = StandardScaler()

# TODO: No hay una manera mejor de hacer esto?
keysOccupation  = unionColumnas(X, Y, "Occupation")
keysCreditMix   = unionColumnas(X, Y, "Credit_Mix")
keysPMA         = unionColumnas(X, Y, "Payment_of_Min_Amount")
keysAIM         = unionColumnas(X, Y, "Payment_Behaviour")

# Transformar las columnas
# TODO: Considerar ensembles para mismos clientes?
# TODO: Considerar OrdinalEncoder para features del estilo Good-Normal-Bad
transformersX = [
  ("ID",                                              "drop",                                                    [0]),    # Por razones obvias
  ("Customer_ID",                                     "drop",                                                    [1]),    # Por razones obvias
  ("Month",                                           "drop",                                                    [2]),    # No creo que tenga ninguna influencia
  ("Name",                                            "drop",                                                    [3]),    # No tiene influencia y está repetida
  ("Age",                                             normalizer,                                                [4]),
  ("SSN",                                             "drop",                                                    [5]),    # No tiene influencia y está repetida
  ("Occupation",                                      OneHotEncoder(categories=[keysOccupation]),                [6]),
  ("Annual_Income",                                   normalizer,                                                [7]),
  ("Monthly_Inhand_Salary",                           "drop",                                                    [8]),    # Annual_Income/12
  ("Num_Bank_Accounts",                               "drop",                                                    [9]),    # No tiene influencia
  ("Num_Credit_Card",                                 normalizer,                                                [10]),
  ("Interest_Rate",                                   normalizer,                                                [11]),
  ("Num_of_Loan",                                     normalizer,                                                [12]),
  ("Type_of_Loan",                                    "drop",                                                    [13]),   # Dividida
  ("Delay_from_due_date",                             normalizer,                                                [14]),
  ("Num_of_Delayed_Payments",                         normalizer,                                                [15]),
  ("Changed_Credit_Limit",                            normalizer,                                                [16]),
  ("Num_Credit_Inquiries",                            normalizer,                                                [17]),
  ("Credit_Mix",                                      OneHotEncoder(categories=[keysCreditMix]),                 [18]),
  ("Outstanding_Debt",                                normalizer,                                                [19]),
  ("Credit_Utilization_Ratio",                        normalizer,                                                [20]),
  ("Credit_History_Age",                              normalizer,                                                [21]),
  ("Payment_of_Min_Amount",                           OneHotEncoder(categories=[keysPMA]),                       [22]),   # TODO: Considerar binary encoding
  ("Total_EMI_per_Month",                             normalizer,                                                [23]),
  ("Amount_Invested_Monthly",                         "drop",                                                    [24]),   # No tiene influencia
  ("Payment_Behaviour",                               OneHotEncoder(categories=[keysAIM]),                       [25]),
  ("Monthly_Balance",                                 normalizer,                                                [26]),
  ("Credit_Score",                                    "drop",                                                    [27]),
  
  #####################
  # Columnas añadidas #
  #####################
  
  # Type_of_Loan
  ("_Auto Loans",                                     normalizer,                                              [28]),
  ("_Credit-Builder Loans",                           normalizer,                                              [29]),
  ("_Debt Consolidation Loans",                       normalizer,                                              [30]),
  ("_Home Equity Loans",                              normalizer,                                              [31]),
  ("_Mortgage Loans",                                 normalizer,                                              [32]),
  ("_Payday Loans",                                   normalizer,                                              [33]),
  ("_Personal Loans",                                 normalizer,                                              [34]),
  ("_Student Loans",                                  normalizer,                                              [35]),
  ("_Not Specified Loans",                            normalizer,                                              [36]),
]

transformersY = deepcopy(transformersX)

# Normbrar el orden de columnas de 0 a n-1
transformersY.pop(27) # Eliminar precio
for i, t in enumerate(transformersY): # Renumerar los indices de las columnas
  transformersY[i][2].pop()
  transformersY[i][2].append(i)

X_T = ColumnTransformer(transformers=transformersX).fit_transform(X)
Y_T = ColumnTransformer(transformers=transformersY).fit_transform(Y)

---

<h3><center>Score</center></h3>

---

In [8]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# 0.72405   / 0.004890080840333814
# 0.7248625 / 0.0027706610222111793

if IS_TEST:
  scores = cross_val_score(clf, X_T, X["Credit_Score"], cv=5, scoring="f1_micro") # Stack Overflow dice que f1_micro es mejor para datasets inbalanceados

  print("Scores :", scores)
  print("F1 avg :", stat.mean(scores))
  print("F1 std :", stat.variance(scores)**(1/2)) # Esto significa algo para F1?

Scores : [0.80625   0.80475   0.8101875 0.8068125 0.804    ]
F1 avg : 0.8064
F1 std : 0.0023989743902342268


In [ ]:
clf = RandomForestClassifier(n_estimators=100, max_depth = 10, random_state=42)

parameters = {
  
  'n_estimators': range(70,100,5),
  'max_depth': range(30,45,5),
  'random_state' : range(0,1000,50)
  
}


gs = GridSearchCV(clf, parameters, verbose=1, n_jobs=-1, cv=5, scoring = "f1_micro")

gs.fit(X_T, X["Credit_Score"])

pd.DataFrame(gs.cv_results_).sort_values('rank_test_score')

Fitting 5 folds for each of 360 candidates, totalling 1800 fits


---

<h3><center>Entrenamiento</center></h3>

---

In [ ]:
if not IS_TEST:
  clf.fit(X_T, X["Credit_Score"])

---

<h3><center>Predicción</center></h3>

---

In [ ]:
pred = None

if not IS_TEST:
  pred = clf.predict(Y_T)


---

<h3><center>CSV</center></h3>

---


In [ ]:
solucion = None

if not IS_TEST:
  solucion = pd.DataFrame(data = { "ID" : Y["ID"], "Credit_Score": pred })
  display(solucion)

In [ ]:
if not IS_TEST:
  solucion.to_csv("solucion.csv", index = False)